In [1]:
# Load Dataset
from sklearn.datasets import load_wine

In [2]:
wine_data = load_wine()

In [3]:
type(wine_data)

sklearn.utils.Bunch

In [4]:
wine_data.keys()

dict_keys(['DESCR', 'target_names', 'feature_names', 'data', 'target'])

In [5]:
print(wine_data['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [6]:
feat_data = wine_data['data']

In [7]:
labels = wine_data['target']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size=0.3, random_state=0)

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scaled_x_train = scaler.fit_transform(X_train)

In [13]:
scaled_x_test = scaler.transform(X_test)

## Using TF-Layers API

In [14]:
import tensorflow as tf
import pandas as pd

c:\users\hare krishna\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
oneHot_y_train = pd.get_dummies(y_train).as_matrix()

In [16]:
oneHot_y_test = pd.get_dummies(y_test).as_matrix()

In [17]:
num_features = 13
hidden1 = 13
hidden2 = 13
output_layer = 3
lr = 0.01

In [18]:
from tensorflow.contrib.layers import fully_connected

In [19]:
X = tf.placeholder(tf.float32, shape=[None, num_features])
y = tf.placeholder(tf.float32, shape=[None,output_layer])

In [20]:
activation_fn = tf.nn.relu

In [21]:
hidden_layer1 = fully_connected(X, hidden1, activation_fn=activation_fn)
hidden_layer2 = fully_connected(hidden_layer1, hidden2, activation_fn=activation_fn)
output = fully_connected(hidden_layer2, output_layer) 

In [22]:
# Loss Function
loss = tf.losses.softmax_cross_entropy(onehot_labels=y, logits = output)
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [23]:
init = tf.global_variables_initializer()

In [24]:
train_steps = 3

In [25]:
# Save Trained Model
save_model = tf.train.Saver()

In [26]:
with tf.Session() as sess:
    sess.run(init)
    
    for steps in range(train_steps):
        sess.run(optimizer, feed_dict={X:scaled_x_train, y: oneHot_y_train})
    
    logits = output.eval(feed_dict={X:scaled_x_test})
    
    # Get value with highest probability prediction
    preds = tf.argmax(logits, axis=1)
    
    results = preds.eval()
    
    save_model.save(sess, './tf-layers-saved-model/tf-layers-trained-model')

In [27]:
from sklearn.metrics import classification_report

In [28]:
print(classification_report(results, y_test))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         9
          1       0.95      0.47      0.63        45
          2       0.00      0.00      0.00         0

avg / total       0.80      0.39      0.52        54



c:\users\hare krishna\appdata\local\programs\python\python35\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
